In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import seaborn as sns
import pandas as pd

sns.set_theme()
sns.set(context="notebook", font="Verdana", font_scale=1.5)

matplotlib.rcParams['figure.figsize'] = (16, 9)
matplotlib.rcParams['figure.dpi'] = 200
matplotlib.rcParams['lines.linewidth'] = 2.5
print("Default figure size:", matplotlib.rcParams['figure.figsize'])

In [ ]:
hourly_emissions_2021 = pd.read_csv('NL_2021_hourly_emissions.csv', index_col="Datetime (UTC)", parse_dates=['Datetime (UTC)'])
hourly_emissions_2022 = pd.read_csv('NL_2022_hourly_emissions.csv', index_col="Datetime (UTC)", parse_dates=['Datetime (UTC)'])

hourly_emissions = pd.concat([hourly_emissions_2021, hourly_emissions_2022])
hourly_emissions.drop(columns=["Country", "Zone Name", "Zone Id", "Data Source", "Data Estimated", "Data Estimation Method"], inplace=True)
hourly_emissions.index = pd.to_datetime(hourly_emissions.index, utc=True)
hourly_emissions.index = hourly_emissions.index.tz_convert('Europe/Amsterdam')
hourly_emissions.index.name = "Datetime (Europe/Amsterdam)"

hourly_emissions.rename(
    columns={
        "Renewable Percentage": "Renewable percentage (Electricity Maps)",
    },
    inplace=True,
)

hourly_emissions = hourly_emissions.resample('15min').interpolate(method='linear')

hourly_emissions.head()

In [ ]:
spring_2021 = pd.read_csv('raw_data/2021_spring_raw_data.csv', parse_dates=[0], index_col=0)
spring_2021["Season"] = "Spring"

summer_2021 = pd.read_csv('raw_data/2021_summer_raw_data.csv', parse_dates=[0], index_col=0)
summer_2021["Season"] = "Summer"

fall_2021 = pd.read_csv('raw_data/2021_fall_raw_data.csv', parse_dates=[0], index_col=0)
fall_2021["Season"] = "Fall"

winter_2021 = pd.read_csv('raw_data/2021_winter_raw_data.csv', parse_dates=[0], index_col=0)
winter_2021["Season"] = "Winter"

df_2021 = pd.concat([spring_2021, summer_2021, fall_2021, winter_2021])
df_2021.index = pd.to_datetime(df_2021.index, utc=True).tz_convert('Europe/Amsterdam')
print("Datatype of df_2021 index:", df_2021.index.dtype)
print(df_2021.index[0])

df_2021["Total renewable generation"] = df_2021["Solar"] + df_2021["Wind Offshore"] + df_2021["Wind Onshore"]
df_2021["Renewable percentage"] = df_2021["Total renewable generation"] / df_2021["Forecasted Load"] * 100

df_2021.head()

In [ ]:
print("Datatype of hourly_emissions:", hourly_emissions.index.dtype)
print("Datatype of df_2021:", df_2021.index.dtype)
merged = pd.merge_asof(df_2021, hourly_emissions, left_index=True, right_index=True, direction="nearest")

merged["Hour"] = merged.index.hour
merged["Minute"] = merged.index.minute

merged

In [ ]:
reg_results = {}
for season in ["Spring", "Summer", "Fall", "Winter"]:
    test_df = merged[merged["Season"] == season]

    y = test_df["Carbon Intensity gCO₂eq/kWh (LCA)"]
    x = test_df["Renewable percentage"].apply(lambda the_x: np.log(the_x))
    
    x = sm.add_constant(x)
    
    model = sm.OLS(y, x)
    res = model.fit()
    
    reg_results[season] = res
    
    print("Season:", season)
    print(res.summary())
    print("\n\n\n\n\n\n")
    
for season_key in reg_results:
    print("Season:", season_key)
    res = reg_results[season_key]
    print(res.params)
    print(res.t_test([0, 1]))
    print("\n")

In [ ]:
g = sns.FacetGrid(
    data=merged, 
    col="Season", 
    height=6, 
    aspect=1, 
    col_wrap=2
)

g.map_dataframe(
    sns.scatterplot,
    x="Renewable percentage",
    y="Carbon Intensity gCO₂eq/kWh (LCA)",
    alpha=0.3,
)

for ax in g.axes.flatten():
    season = ax.get_title().split(" = ")[1]
    x_plot = np.linspace(0, 100, 100)
    const = reg_results[season].params['const']
    coeff = reg_results[season].params['Renewable percentage']
    y_plot = const + coeff * np.log(x_plot)
    
    sns.lineplot(x=x_plot, y=y_plot, ax=ax, label=f"y = {round(coeff, 4)} ln(x) + {round(const, 4)}", color="red")
    
    

In [ ]:
g = sns.FacetGrid(
    data=merged, 
    col="Season", 
    height=6, 
    aspect=1, 
    col_wrap=2
)

g.map_dataframe(
    sns.scatterplot,
    x="Renewable percentage (Electricity Maps)",
    y="Carbon Intensity gCO₂eq/kWh (LCA)",
    alpha=0.3,
)

# for ax in g.axes.flatten():
#     season = ax.get_title().split(" = ")[1]
#     x_plot = np.linspace(0, 100, 100)
#     const = reg_results[season].params['const']
#     coeff = reg_results[season].params['Renewable percentage']
#     y_plot = const + coeff * x_plot
#     
#     sns.lineplot(x=x_plot, y=y_plot, ax=ax, label=f"y = {round(coeff, 4)}x + {round(const, 4)}", color="red")
    
    

In [ ]:
fig, ax = plt.subplots()

sns.scatterplot(merged, x="Renewable percentage (Electricity Maps)", y="Renewable percentage", alpha=0.3, ax=ax)